<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/Colab%20copia%20NLP_02_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico NLP - Detección de Tópicos y clasificación
- ITBA 2024
- Alumno: Gabriel Rey
---

### NLP_02_model

Esta notebook se utiliza para:
- armar el modelo de estimación de tópicos
- validar resultados obtenidos
- guardar datos en los indices de opensearch

Esta y las consecuentes notebooks son el desarrollo de base de procesos y funciones para la web app provista.

### MODELO

In [1]:
%%capture
from google.colab import userdata

git_token = userdata.get('GIT_TOKEN')
git_username = "gabrielfernandorey"
git_repository = "ITBA-NLP.git"

!git clone https://{git_token}@github.com/{git_username}/{git_repository}

In [2]:
%%capture
!pip install python-dotenv
!pip install bertopic
!pip install OpenAI
!pip install datasets

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
from matplotlib import pyplot as plt
import os, sys
import json
import pickle
from datetime import datetime, date
from dateutil.parser import parse
from dotenv import load_dotenv

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF

from tqdm import tqdm

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

from openai import OpenAI
from datasets import load_dataset

In [4]:
import unicodedata
from functools import wraps
import re
class Cleaning_text:
    '''
    Limpiar elementos no deseados del texto
    '''

    def __init__(self):
        # Definir los caracteres Unicode no deseados
        self.unicode_pattern    = ['\u200e', '\u200f', '\u202a', '\u202b', '\u202c', '\u202d', '\u202e', '\u202f']
        self.urls_pattern       = re.compile(r'http\S+')
        self.simbols_chars      = r"""#&’'"`´“”″()[]*+,-.;:/<=>¿?!¡@\^_{|}~©√≠"""                 # Lista de símbolos a eliminar
        self.simbols_pattern    = re.compile(f"[{re.escape(self.simbols_chars)}]")
        self.escape_pattern     = ['\n', '\t', '\r']

    def _clean_decorator(clean_func):
        @wraps(clean_func)
        def wrapper(self, input_data):
            def clean_string(text):
                return clean_func(self, text)

            if isinstance(input_data, str):
                return clean_string(input_data)
            elif isinstance(input_data, list):
                return [clean_string(item) for item in input_data]
            else:
                raise TypeError("El argumento debe ser una cadena o una lista de cadenas.")
        return wrapper

    @_clean_decorator
    def unicode(self, text):
        for pattern in self.unicode_pattern:
            text = text.replace(pattern, ' ')
        return text

    @_clean_decorator
    def urls(self, text):
        return self.urls_pattern.sub(' ', text)

    @_clean_decorator
    def simbols(self, text):
        return self.simbols_pattern.sub(' ', text)

    @_clean_decorator
    def accents_emojis(self, text):
        return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')

    @_clean_decorator
    def escape_sequence(self, text):
        for pattern in self.escape_pattern:
            text = text.replace(pattern, ' ').strip()
        return text

    @_clean_decorator
    def str_lower(self, text):
        return text.lower()

#---------------------------------------------------------------------------------------------------------
def clean_all(entities, accents=True, lower=True) -> list:
    """
    Función que toma una lista de entidades, realiza una operación de limpieza
    y devuelve una lista de entidades limpias.
    """
    cleaner = Cleaning_text()

    entities_clean = []
    for ent in entities:
        clean_txt = cleaner.unicode(ent)
        clean_txt = cleaner.urls(clean_txt)
        clean_txt = cleaner.simbols(clean_txt)

        if accents:
            clean_txt = cleaner.accents_emojis(clean_txt)

        clean_txt = cleaner.escape_sequence(clean_txt)

        if lower:
            clean_txt = cleaner.str_lower(clean_txt)

        entities_clean.append(" ".join(clean_txt.split()))

    return entities_clean

### Path

In [5]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

'/content/ITBA-NLP/data/'

In [6]:
if PATH == os.environ.get('PATH_LOCAL'):
    if os.environ.get('OPENAI_API_KEY'):
        client = OpenAI(api_key= os.environ.get('OPENAI_API_KEY'))
    else:
        client = None
    print(client)

### Data de noticias original

In [7]:
def get_data_news_parquet(date_choice):
    # Desde Hugginface
    path_file = f"jganzabalseenka/news_{date_choice}_24hs"
    dataset = load_dataset(path_file)
    df_parquet = pd.DataFrame(dataset['train'])
    return df_parquet

def valid_data_news(df_parquet, date_choice):
    # Validar datos para la fecha
    choice = "".join(date_choice.split('-'))
    df_parquet.sort_values("start_time_local", ascending=True, inplace=True)
    df_date_filtered = df_parquet[df_parquet['start_time_local'].dt.date == pd.to_datetime(choice).date()]
    print(f"Registros para la fecha {choice} -> {len(df_date_filtered)} de un total de {len(df_parquet)}")
    return df_date_filtered

def get_batch_news(df_date_filtered, batch_news):
    # Obtenemos un lote de N noticias por dia (para agilizar el procesamiento)
    df_batch = df_date_filtered.sample(n=int(batch_news)).reset_index()
    data_1 = list(df_batch['text'])
    id_data_1 = list(df_batch['asset_id'])
    title_data_1 = list(df_batch['title'])
    print(f"Batch: {len(data_1)}")
    return data_1, id_data_1, title_data_1

def clean_data_news(data):
    clean_data = Cleaning_text()
    proc_data = []
    for data_in in tqdm(data):
        aux = clean_data.unicode(data_in)
        aux = clean_data.urls(aux)
        aux = clean_data.simbols(aux)
        aux = clean_data.escape_sequence(aux)
        aux = " ".join([ word for word in aux.split() if word.lower() not in SPANISH_STOPWORDS_SPECIAL])
        proc_data.append(aux)
    return proc_data

### Preprocesar las noticias
Se realiza un preprocesamiento mínimo del texto, pero no se le quita el sentido semántico para que mediante SentenceTransformer se puedan capturar embeddings de mejor calidad.

In [8]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SPECIAL = list(pd.read_csv(PATH+'spanish_stop_words_spec.csv' )['stopwords'].values)

### Modelo

In [45]:
#### OnLine Topic Modeling
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import IncrementalPCA
from bertopic.vectorizers import OnlineCountVectorizer

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Prepare sub-models that support online learning
umap_model = IncrementalPCA(n_components=5)
cluster_model = MiniBatchKMeans(n_clusters=50, random_state=42)
vectorizer_model = OnlineCountVectorizer(stop_words=SPANISH_STOPWORDS, decay=.01)

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

topic_online_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=cluster_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    verbose=True
    )

### Entrenamiento

In [46]:
titles = []
topics = []
total_data = []

In [56]:
dates_list = ['2024-07-17']
batch_news = 1000
for date_choice in dates_list:
    df_parquet = get_data_news_parquet(date_choice)
    df_date_filtered = valid_data_news(df_parquet, date_choice)
    data, id_data, title_data = get_batch_news(df_date_filtered, batch_news)

    titles.extend(title_data)
    total_data.extend(data)

    proc_data = clean_data_news(data)
    topics_online = topic_online_model.partial_fit(proc_data)
    topics.extend(topic_online_model.topics_)



Registros para la fecha 20240717 -> 11663 de un total de 16037
Batch: 1000


100%|██████████| 1000/1000 [00:04<00:00, 201.05it/s]


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-25 19:25:01,678 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-25 19:25:02,526 - BERTopic - Dimensionality - Completed ✓
2024-08-25 19:25:02,532 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-25 19:25:02,538 - BERTopic - Cluster - Completed ✓


In [61]:
topic_id = np.random.randint(len(set(topic_online_model.topics_))-1) # Obtener un topico al azar
topic_id = 48
topic_keywords = topic_online_model.get_topics()[topic_id]
docs_position_per_topic = [i for i, x in enumerate(topics) if x == topic_id]
print(topic_id, "_".join([keyword[0] for keyword in topic_keywords[:4]]))
print('-'*80)
for doc in docs_position_per_topic:
    print(doc, titles[doc])

48 copa_selección_américa_argentina
--------------------------------------------------------------------------------
23 Deportes
121 Caro Calvagni, la esposa de Nico Tagliafico, contó cómo fue la romántica propuesta de casamiento
220 La Federación Francesa denunciaría a los jugadores de la Selección argentina por cantos racistas y homofóbicos
260 La Federación Francesa denunciaría a los jugadores de la Selección argentina por cantos racistas y homofóbicos
266 La Selección trajo la Copa América a Argentina
290 "La versión malvada de Julián Álvarez": el increíble parecido del hermano del delantero
350 El conmovedor elogio a Messi de un comentarista brasileño luego de la escena del llanto: "La imagen es muy poderosa"
429 El particular elogio de un exjugador colombiano a la Scaloneta: "En Argentina el segundo se pone a llorar"
436 Video: en medio de cantos eufóricos, así llegaba la Selección argentina tras ganar la final de la Copa América
504 La Selección trajo la Copa América a Argentina

In [57]:


topic_counts = {}
for topic in set(topics):
  topic_counts[topic] = topics.count(topic)

topic_counts_list = [[topic, count] for topic, count in topic_counts.items()]
print(len(topic_counts_list))
for i in range(len(topic_counts_list)):
    print(topic_counts_list[i])


50
[0, 66]
[1, 71]
[2, 47]
[3, 43]
[4, 44]
[5, 31]
[6, 43]
[7, 35]
[8, 55]
[9, 16]
[10, 31]
[11, 34]
[12, 36]
[13, 30]
[14, 72]
[15, 21]
[16, 40]
[17, 58]
[18, 7]
[19, 34]
[20, 35]
[21, 33]
[22, 57]
[23, 19]
[24, 57]
[25, 58]
[26, 54]
[27, 42]
[28, 28]
[29, 24]
[30, 37]
[31, 42]
[32, 22]
[33, 69]
[34, 16]
[35, 9]
[36, 43]
[37, 67]
[38, 37]
[39, 29]
[40, 34]
[41, 48]
[42, 23]
[43, 30]
[44, 32]
[45, 36]
[46, 77]
[47, 44]
[48, 25]
[49, 59]


In [63]:
embeddings = topic_online_model.embedding_model.embed(total_data)

In [64]:
def calculate_probabilities(model, docs):
    # Transformar documentos al espacio vectorial
    embeddings = model.embedding_model.embed(docs)

    # Aplicar reducción de dimensionalidad
    reduced_embeddings = model.umap_model.transform(embeddings)

    # Predecir clústeres y calcular distancias
    clusters = model.hdbscan_model.predict(reduced_embeddings)
    distances = model.hdbscan_model.transform(reduced_embeddings)

    # Aproximar probabilidades (probabilidad inversamente proporcional a la distancia)
    max_distances = np.max(distances, axis=1, keepdims=True)
    probs = 1 - (distances / max_distances)
    return clusters, probs

In [65]:
# Obtener probabilidades para un nuevo conjunto de datos
topics_x, probs_x = calculate_probabilities(topic_online_model, data)

In [66]:
def topic_data_view(model, df, probs=None):
    reg = []
    for topic_number in range(len(model.get_topic_info())):
        docs_per_topics = [i for i, x in enumerate(model.topics_) if x == topic_number]
        for doc in docs_per_topics:
            if probs is None:
                reg.append([str(topic_number), str(doc), df.iloc[doc].title])
                columns=['Topic','Id Doc','Title']
            else:
                prob = np.max(probs[doc])
                reg.append([str(topic_number), str(doc), df.iloc[doc].title, prob])
                columns=['Topic','Id Doc','Title','Probs']

    df_query = pd.DataFrame(reg, columns=columns)
    return df_query

In [67]:
topic_data_view(topic_online_model, df_parquet, probs_x)

,Topic,Id Doc,Title,Probs
0,0,0,Venado Tuerto: Joven Herido de Gravedad tras Violenta Riña,0.835167
1,0,22,Empleo remoto y pagan 48.000 dólares al año: cómo aplicar,0.775971
2,0,89,En el Servicio Penitenciario de San Juan ayudan a jóvenes que transitan su primera condena,0.836286
3,0,98,"Estados Unidos y China enfrentados por la supremacía del dinero cuántico, un nuevo paradigma mundial",0.803121
4,0,106,Walter Mercado: Horóscopos mágicos de la semana del 16 al 22 de julio,0.808945
...,...,...,...,...
995,49,926,Viví las vacaciones de invierno en Baradero,0.749698
996,49,927,PreocupaciónConfirmado por Inter Miami: ¿Qué lesión tiene Messi y cuánto tiempo de recuperación?,0.817684
997,49,966,"A un año de ser diagnosticada con leucemia, Wanda Nara compartió una impactante foto de su internación",0.737150
998,49,983,«¿Cuál es la diferencia entre el euro hoy y el euro blue? Descubre cómo cotizaron este martes 16 de julio»,0.712060
